In [1]:
import isaacgym
import isaacgymenvs
from isaacgymenvs.utils.reformat import omegaconf_to_dict, print_dict
from isaacgymenvs.utils.utils import set_np_formatting, set_seed
from isaacgymenvs.utils.rlgames_utils import RLGPUEnv, RLGPUAlgoObserver, get_rlgames_env_creator

from rl_games.common import env_configurations, vecenv
from rl_games.torch_runner import Runner
from rl_games.algos_torch import model_builder

from omegaconf import DictConfig, OmegaConf

import torch
import numpy as np
import matplotlib.pyplot as plt

from vecrobotics import *
from fista import QP, FISTA

Importing module 'gym_38' (/home/hz/devel/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Setting GYM_USD_PLUG_INFO_PATH to /home/hz/devel/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json


PyTorch version 2.0.0
Device count 1
/home/hz/devel/isaacgym/python/isaacgym/_bindings/src/gymtorch


Using /home/hz/.cache/torch_extensions/py38_cu117 as PyTorch extensions root...
Emitting ninja build file /home/hz/.cache/torch_extensions/py38_cu117/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module gymtorch...
2023-05-11 18:08:25,929 - INFO - logger - logger initialized
<unknown>:3: DeprecationWarning: invalid escape sequence \*
/home/hz/miniforge3/envs/rlgpu/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/home/hz/miniforge3/envs/rlgpu/lib/python3.8/site-packages/matplotlib/__init__.py:169: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
/home/hz/miniforge3/envs/rlgpu/lib/python3.8/site-packages/setuptools/_distutils/version.py:345: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


Error: FBX library failed to load - importing FBX data will not succeed. Message: No module named 'fbx'
FBX tools must be installed from https://help.autodesk.com/view/FBX/2020/ENU/?guid=FBX_Developer_Help_scripting_with_python_fbx_installing_python_fbx_html


In [2]:
cfg = OmegaConf.load("../isaacgymenvs/cfg/config.yaml")
cfg.task_name = "TrifingerNYU"
cfg.num_envs = 1
cfg.task = OmegaConf.load("../isaacgymenvs/cfg/task/TrifingerNYU.yaml")
cfg.headless = True

In [3]:
def create_env_thunk(**kwargs):
    envs = isaacgymenvs.make(
            cfg.seed, 
            cfg.task_name, 
            cfg.task.env.numEnvs, 
            cfg.sim_device,
            cfg.rl_device,
            cfg.graphics_device_id,
            cfg.headless,
            cfg.multi_gpu,
            cfg.capture_video,
            cfg.force_render,
            cfg,
            **kwargs,
        )
    return envs

In [4]:
device = cfg.sim_device
envs = create_env_thunk()

/home/hz/miniforge3/envs/rlgpu/lib/python3.8/site-packages/gym/spaces/box.py:112: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/hz/learning/IsaacGymEnvs/isaacgymenvs/tasks/trifingerNYU.py:1278: DeprecationWarning: an integer is required (got type isaacgym._bindings.linux-x86_64.gym_38.DofDriveMode).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  robot_asset_options.default_dof_drive_mode = gymapi.DOF_MODE_EFFORT


[Warning] [carb.gym.plugin] useGpu is set, forcing single scene (0 subscenes)
Not connected to PVD
+++ Using GPU PhysX
Physics Engine: PhysX
Physics Device: cuda:0
GPU Pipeline: enabled
Trifinger Robot Asset: 
	 Number of bodies: 23
	 Number of shapes: 39
	 Number of dofs: 9
	 Number of actuated dofs: 9
Trifinger Table Asset: 
	 Number of bodies: 2
	 Number of shapes: 1
Trifinger Boundary Asset: 
	 Number of bodies: 1
	 Number of shapes: 41
Using VHACD cache directory '/home/hz/.isaacgym/vhacd'
Found existing convex decomposition for mesh '/home/hz/learning/IsaacGymEnvs/isaacgymenvs/../assets/trifinger/robot_properties_fingers/meshes/pro/int_sim.stl'
Found existing convex decomposition for mesh '/home/hz/learning/IsaacGymEnvs/isaacgymenvs/../assets/trifinger/robot_properties_fingers/meshes/pro/prox-sim.stl'
Found existing convex decomposition for mesh '/home/hz/learning/IsaacGymEnvs/isaacgymenvs/../assets/trifinger/robot_properties_fingers/meshes/pro/tip_link_sim.stl'
Found existing co

In [5]:
lifting_data = np.load("data/lifting.npz", allow_pickle=True)["data"]

In [6]:
# get fingertip states
envs.reset_idx(torch.arange(cfg.num_envs))
N = 570
action_buffer = torch.zeros(N, 9).to(device)
ftip_pos_buffer = torch.zeros(N, 3, 3).to(device)
ftip_pos_local_buffer = torch.zeros(N, 3, 3).to(device)
object_pose_buffer = torch.zeros(N, 7).to(device)

q_buffer = torch.zeros(N, 9).to(device)
dq_buffer = torch.zeros(N, 9).to(device)
obs, rwds, resets, info = envs.step(torch.zeros(cfg.num_envs, 9))

In [7]:
for n in range(N):
    q = envs._dof_position
    dq = envs._dof_velocity
    q_buffer[n] = q[0]
    dq_buffer[n] = dq[0]
    
    fingertip_state = envs._rigid_body_state[:, envs._fingertip_indices]
    fingertip_position = fingertip_state[:, :, 0:3]
    ftip_pos_buffer[n] = fingertip_position[0]
    
    object_pose = envs._object_state_history[0][:, 0:7]
    object_pose_buffer[n] = object_pose[0]

    for i in range(3):
        ftip_pos_local_buffer[n, i] = world2local(object_pose, fingertip_position[:, i, :])
        
        
#     object_pos = object_pose[:, 0:3]
#     pos_diff = object_pos.repeat(1, 3) - fingertip_position.reshape(cfg.num_envs, 9)
    
    ftip_pos_des = torch.tensor(lifting_data[20 * n]['policy']['controller']['ft_pos_des'], dtype=torch.float32).to(device)
    action = ftip_pos_des.view(cfg.num_envs, 9) - fingertip_position.reshape(cfg.num_envs, 9)
     
    action_buffer[n] = action[0]
    obs, rwds, resets, info = envs.step(action)
    

In [8]:
def get_cube_contact_normals(ftip_pos, threshold=0.0435):
    batch_size = len(ftip_pos)
    contact_normals = torch.zeros(batch_size, 3).to(ftip_pos.device)
        
    _, max_indices = torch.max(torch.abs(ftip_pos), dim=1)
    max_values = torch.squeeze(torch.gather(ftip_pos, 1, max_indices.unsqueeze(1)))

    mask_pos = (torch.abs(max_values) <= threshold) * (max_values > 0)
    mask_neg = (torch.abs(max_values) <= threshold) * (max_values < 0)

    # contact normal points to the same direction as the contact force, hence into the object
    contact_normals[mask_pos, max_indices[mask_pos]] = -1.0
    contact_normals[mask_neg, max_indices[mask_neg]] = 1.0
    
    return contact_normals

def get_contact_frame_orn(contact_normals: torch.Tensor):
    # get the orientation of the contact frames expressed in the object frame
    z_axis = contact_normals
    zero_indices = torch.argmax(torch.eq(z_axis, 0).int(), dim=1)
    y_axis = torch.eye(3).to(z_axis.device)[zero_indices]
    x_axis = torch.cross(y_axis, z_axis)
    y_axis = torch.cross(z_axis, x_axis) # this makes sure if z is all zero, then orn is a zero matrix
    orn = torch.stack((x_axis, y_axis, z_axis), dim=2)
    return orn

In [62]:
num_batches, _ = object_pose_buffer.shape
num_vars = 9
num_eqc = 1
A = torch.zeros(num_batches, num_eqc, num_vars)
b = torch.zeros(num_batches, num_eqc)
rho = 0 # rho = 0 supresss eqc Ax + b = 0
lb = -10 * torch.ones(num_batches, num_vars)
ub = 10 * torch.ones(num_batches, num_vars)

In [63]:
ftip_pos = ftip_pos_local_buffer.view(-1, 3)
contact_normals = get_cube_contact_normals(ftip_pos)

In [64]:
orn = get_contact_frame_orn(contact_normals)
orn_stacked = orn.transpose(1, 2).reshape(-1, 9, 3)

In [65]:
Q = orn_stacked @ orn_stacked.transpose(1, 2)

# prevents nan for fingers that are not in contact
diag_idx = torch.arange(num_vars)
Q[:, diag_idx, diag_idx] = 1

In [66]:
g = torch.tensor([0, 0, 9.81]).repeat(num_batches, 1).to(device)
object_orn = quat2mat(object_pose_buffer[:, 3:])
g_local = bmv(object_orn, g)
m = 1

In [67]:
q = -2 * m * bmv(orn_stacked, g_local)

In [68]:
prob = QP(num_batches, num_vars, num_eqc, friction_coeff=1.0, device=device)
prob.set_data(Q, q, A, b, rho, lb, ub)
solver = FISTA(prob, device=device)
max_it = 100

In [69]:
for i in range(max_it):
    solver.step()

In [70]:
F = solver.prob.yk

In [71]:
F_ = bmv(orn, F.view(num_batches*3, 3))

In [72]:
i = 115

In [73]:
F_[i*3:(i+1)*3]

tensor([[0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000],
        [4.9050, 0.0000, 0.0000]], device='cuda:0')

In [74]:
orn[i*3:(i+1)*3]

tensor([[[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],

        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],

        [[ 0., -0.,  1.],
         [ 0.,  1.,  0.],
         [-1.,  0.,  0.]]], device='cuda:0')

In [75]:
contact_normals[i*3:(i+1)*3]

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [1., 0., 0.]], device='cuda:0')

In [76]:
f_total = bmv(orn_stacked.transpose(1, 2), F)